# `nx_merge_parallel_edges()` with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

-----------------

1. Setup
2. Prep work
3. Experiment

-----------------

## 1. Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-04-18T14:22:12.706164-04:00

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import cityseer  # noqa: F401
import momepy
import utils
from cityseer.tools import graphs, io

%watermark -w
%watermark -iv

Watermark: 2.4.3

cityseer: 4.12.0
momepy  : 0.7.1.dev29+g593fc8e



-----------------------------------------
## 1. Setup

* https://cityseer.benchmarkurbanism.com/tools/graphs#nx-merge-parallel-edges

In [3]:
# city = "Douala"
city = "Liège"
utils.city_fua[city]

1656

In [4]:
roads_bare = utils.read_parquet_roads(utils.city_fua[city])
crs = roads_bare.crs
# roads_bare.to_crs(4326).to_file("TEMP_DOUALA.geojson")
# roads_bare.to_crs(4326).to_file("TEMP_LIEGE.geojson")

## Prep `cityseer` graph
* performs an initial 1-meter parallel edge buffer

In [5]:
G = io.nx_from_generic_geopandas(roads_bare)
print(utils.graph_size(f"NetworkX Graph from CitySeer: {city}", G))

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105174/105174 [00:00<00:00, 364363.90it/s]

NetworkX Graph from CitySeer: Liège
	* MultiGraph with 98777 nodes and 105156 edges


In [6]:
print(help(graphs.nx_merge_parallel_edges))

Help on function nx_merge_parallel_edges in module cityseer.tools.graphs:

nx_merge_parallel_edges(nx_multigraph: 'MultiGraph', merge_edges_by_midline: 'bool', contains_buffer_dist: 'int') -> 'MultiGraph'
    Check a MultiGraph for duplicate edges; which, if found, will be merged.
    
    The shortest of these parallel edges is selected and buffered by `contains_buffer_dist`. If this buffer contains an
    adjacent edge, then the adjacent edge is merged. Edges falling outside this buffer are retained.
    
    When candidate edges are found for merging, they are replaced by a single new edge. The new geometry selected from
    either:
    - An imaginary centreline of the combined edges if `merge_edges_by_midline` is set to `True`;
    - Else, the shortest edge is retained, with longer edges discarded.
    
    Parameters
    ----------
    nx_multigraph: MultiGraph
        A `networkX` `MultiGraph` in a projected coordinate system, containing `x` and `y` node attributes, and `geom`
  

## Experiement params
* only `merge_edges_by_midline = True`
* `contains_buffer_dists` of 1m (from default) to 100m, by 5m

In [7]:
merge_edges_by_midline = True
contains_buffer_dists = list(range(5, 101, 5))

----------------

## 3. Experiment

In [8]:
buff_dict = {}

# ana's style results recorder
for contains_buffer_dist in contains_buffer_dists:
    buff_dict[contains_buffer_dist] = {}

    # run ``nx_merge_parallel_edges``
    merge_args = G.copy(), merge_edges_by_midline, contains_buffer_dist
    G_merged = graphs.nx_merge_parallel_edges(*merge_args)

    # diagnostic
    _ginfo = f"CitySeer - merged parallel edges\n  {city=} - {contains_buffer_dist=}"
    print(utils.graph_size(_ginfo, G_merged))

    # conversion from ``cityseer`` format to ``momepy``/``geopandas``
    cs_gdf = io.geopandas_from_nx(G_merged, crs=crs).rename_geometry("geometry")
    mm_nx = momepy.gdf_to_nx(cs_gdf, integer_labels=True)
    nodes, edges = momepy.nx_to_gdf(mm_nx)

    nodes = nodes[["nodeID", "geometry"]].copy()
    edges = edges[["node_start", "node_end"]].copy()

    buff_dict[contains_buffer_dist]["nodes"] = nodes
    buff_dict[contains_buffer_dist]["edges"] = edges

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 5.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105156/105156 [00:00<00:00, 335976.54it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


CitySeer - merged parallel edges
  city='Liège' - contains_buffer_dist=5
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 10.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105156/105156 [00:00<00:00, 572882.14it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


CitySeer - merged parallel edges
  city='Liège' - contains_buffer_dist=10
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 15.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105156/105156 [00:00<00:00, 218416.03it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


...

CitySeer - merged parallel edges
  city='Liège' - contains_buffer_dist=90
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 95.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105156/105156 [00:00<00:00, 556163.78it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


CitySeer - merged parallel edges
  city='Liège' - contains_buffer_dist=95
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 100.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 105156/105156 [00:00<00:00, 174346.17it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


CitySeer - merged parallel edges
  city='Liège' - contains_buffer_dist=100
	* MultiGraph with 98777 nodes and 105156 edges


-----------------------------------